# Intiuitive predictions about time series data 
* 02/14/23

In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import json

# local modules 
import sys
sys.path.insert(0, "../scripts")
from helpers import *



## import time series 

In [44]:
def import_desired_data(exp, freq):
    """
    get specific experiment, specific room, and specific frequency of data 

    exp: exp month, A, B, C -> july, aug, sep 
    freq: "60T" = 60 mins, "0.5T" = 1/2 minute

    returns two dataframes split into room 1 and room 0, and optionally for a given experiment, and a given frequency 
    """
    
    # read in the dataframe 
    df = pd.read_csv("../constants/td_ambient_102022.csv" )
    # transform datetime index to actual pandas datetime, 
    df["DateTime"] = pd.to_datetime(df["DateTime"])

    if exp != None:
        # select certain experiments 
        with open('../constants/window_treatment.json') as f:    
            window_treatment = json.load(f)

        exp_a_end = str2dt(window_treatment["072522"]["cutoff_times"]["end"])
        exp_b_end = str2dt(window_treatment["081622"]["cutoff_times"]["end"])

        # HERE branching logic bassed on which experiment considering 
        if exp == "A":
            df_exp = df.loc[df["DateTime"] < exp_a_end]
        elif exp == "B":
            mask = (df['DateTime'] > exp_a_end) & (df['DateTime'] > exp_b_end)
            df_exp = df.loc[mask].reset_index(drop=True)
        elif exp == "C":
            df_exp = df.loc[df["DateTime"] > exp_b_end]
        else:
            df_exp = None

        # split data into dataframes based on room
        try:
            df1, df0 = [x.reset_index(drop=True) for _, x in df_exp.groupby(df_exp['Room'] < 1)]
        except:
            "Invalid experiment name entered!"
            pass

    else:
        df1, df0 = [x.reset_index(drop=True) for _, x in df.groupby(df['Room'] < 1)]

    if freq:
        # filter data based on frequency 
        df00 = make_df_with_freq(df0, freq).droplevel(0).reset_index()
        df01 = make_df_with_freq(df1, freq).droplevel(0).reset_index()

        return df00, df01 
    else:
        return df1, df0


In [46]:
a, b = import_desired_data(None, None)
a


,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
0,2022-07-20 07:20:00,23.453,52.910,1,14.333333,30.555556,0
1,2022-07-20 07:20:30,23.453,52.878,1,14.333333,30.555556,0
2,2022-07-20 07:21:00,23.453,52.847,1,14.333333,30.555556,0
3,2022-07-20 07:21:30,23.453,52.784,1,14.333333,30.555556,0
4,2022-07-20 07:22:00,23.453,52.753,1,14.333333,30.555556,0
...,...,...,...,...,...,...,...
61798,2022-09-11 07:58:00,29.196,50.861,1,19.611111,30.000000,0
61799,2022-09-11 07:58:30,29.196,50.861,1,19.611111,30.000000,0
61800,2022-09-11 07:59:00,29.196,50.796,1,19.611111,30.000000,0
61801,2022-09-11 07:59:30,29.196,50.796,1,19.611111,30.000000,0


In [38]:
a.droplevel(0).reset_index()

,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
0,2022-09-08 08:00:00,29.516625,46.701342,0.0,17.777778,19.444444,0.0
1,2022-09-08 09:00:00,29.859025,47.111300,0.0,22.444444,18.333333,0.0
2,2022-09-08 10:00:00,30.154250,47.461500,0.0,26.277778,15.555556,0.0
3,2022-09-08 11:00:00,30.344475,47.166233,0.0,29.500000,10.000000,0.0
4,2022-09-08 12:00:00,30.388000,45.728700,0.0,32.277778,5.000000,0.0
...,...,...,...,...,...,...,...
68,2022-09-11 04:00:00,28.709617,51.907317,0.0,19.833333,27.777778,0.0
69,2022-09-11 05:00:00,29.023900,53.560967,0.0,19.500000,30.000000,0.0
70,2022-09-11 06:00:00,29.068025,54.782758,0.0,19.500000,30.000000,1.0
71,2022-09-11 07:00:00,28.799642,51.962300,0.0,19.611111,30.000000,1.0


In [22]:
a.reset_index(drop=False)

ValueError: cannot insert Room, already exists

In [15]:
if len(a) > 0:
    print("hi")

hi


In [ ]:
df = pd.read_csv("../constants/td_ambient_102022.csv" )
df["DateTime"] = pd.to_datetime(df["DateTime"])